In [16]:
! pip install triton --force-reinstall

  Using cached triton-3.3.1-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (155.6 MB)
  Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 80.9.0
    Uninstalling setuptools-80.9.0:
      Successfully uninstalled setuptools-80.9.0
  Attempting uninstall: triton
    Found existing installation: triton 3.3.1
    Uninstalling triton-3.3.1:
      Successfully uninstalled triton-3.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.7.0+cu118 requires triton==3.3.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.3.1 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [17]:
# Importando as bibliotecas necessárias
import torch  # Biblioteca para computação tensorial e redes neurais
import triton  # Biblioteca para otimização de operações em GPUs
import triton.language as tl  # Módulo de linguagem específico do Triton para kernels
from triton.runtime import driver  # Módulo para interagir com o driver do Triton
import os  # Módulo para interagir com o sistema operacional

# Função para calcular o softmax de forma ingênua (sem otimização)
def naive_softmax(x):
    # Encontra o valor máximo ao longo das linhas (dim=1)
    x_max = x.max(dim=1)[0]
    
    # Subtrai o valor máximo de cada elemento para estabilidade numérica
    z = x - x_max[:, None]
    
    # Calcula o exponencial de cada elemento
    numerator = torch.exp(z)
    
    # Soma os exponenciais ao longo das linhas para obter o denominador
    denominator = numerator.sum(dim=1)
    
    # Retorna o softmax: exponencial dividido pela soma dos exponenciais
    return numerator / denominator[:, None]

In [18]:
# Decorador para indicar que a função é um kernel Triton JIT (Just-In-Time)
@triton.jit
def softmax_kernel(output_ptr, input_ptr, input_row_stride, output_row_stride, n_rows, n_cols, BLOCK_SIZE: tl.constexpr, num_stages: tl.constexpr):
    # Obtém o índice inicial da linha para o programa atual (thread block)
    row_start = tl.program_id(0)
    # Obtém o número de passos (steps) para processar as linhas
    row_step = tl.num_programs(0)
    
    # Loop para processar as linhas em paralelo
    for row_idx in tl.range(row_start, n_rows, row_step, num_stages=num_stages):
        # Calcula o ponteiro inicial da linha atual na matriz de entrada
        row_start_ptr = input_ptr + row_idx * input_row_stride
        # Cria offsets para as colunas dentro do bloco
        col_offsets = tl.arange(0, BLOCK_SIZE)
        # Calcula os ponteiros para os elementos da linha atual
        input_ptrs = row_start_ptr + col_offsets
        # Cria uma máscara para evitar acessos fora dos limites da matriz
        mask = col_offsets < n_cols
        # Carrega os elementos da linha atual, usando a máscara e substituindo valores inválidos por -inf
        row = tl.load(input_ptrs, mask=mask, other=-float('inf'))
        
        # Subtrai o valor máximo da linha para estabilidade numérica
        row_minus_max = row - tl.max(row, axis=0)
        # Calcula o exponencial dos valores ajustados
        numerator = tl.exp(row_minus_max)
        # Calcula a soma dos exponenciais para normalização
        denominator = tl.sum(numerator, axis=0)
        # Calcula o softmax: exponencial dividido pela soma dos exponenciais
        softmax_output = numerator / denominator
        
        # Calcula o ponteiro inicial da linha atual na matriz de saída
        output_row_start_ptr = output_ptr + row_idx * output_row_stride
        # Calcula os ponteiros para os elementos da linha de saída
        output_ptrs = output_row_start_ptr + col_offsets
        # Armazena o resultado do softmax na matriz de saída, usando a máscara
        tl.store(output_ptrs, softmax_output, mask=mask)



In [19]:
# Define o dispositivo como a primeira GPU disponível
DEVICE = torch.device("cuda:0")

# Obtém as propriedades da GPU ativa
properties = driver.active.utils.get_device_properties(DEVICE.index)
NUM_SM = properties["multiprocessor_count"]  # Número de multiprocessadores (SMs)
NUM_REGS = properties["max_num_regs"]  # Número máximo de registradores por SM
SIZE_SMEM = properties["max_shared_mem"]  # Tamanho máximo da memória compartilhada por SM
WARP_SIZE = properties["warpSize"]  # Tamanho de um warp (32 threads)
target = triton.runtime.driver.active.get_current_target()  # Alvo de compilação atual

# Função para calcular o softmax usando o kernel Triton
def softmax(x):
    # Obtém o número de linhas e colunas da matriz de entrada
    n_rows, n_cols = x.shape
    # Define o tamanho do bloco como a próxima potência de 2 maior que o número de colunas
    BLOCK_SIZE = triton.next_power_of_2(n_cols)
    num_warps = 8  # Número de warps por bloco
    num_stages = 4  # Número de estágios para o pipeline de execução
    # Cria uma matriz de saída vazia com o mesmo formato da entrada
    y = torch.empty_like(x)
    
    # Pré-aquecimento do kernel (compilação e inicialização)
    kernel = softmax_kernel.warmup(y, x, x.stride(0), y.stride(0), n_rows, n_cols, BLOCK_SIZE=BLOCK_SIZE, num_stages=num_stages, num_warps=num_warps, grid=(1,))
    kernel._init_handles()  # Inicializa os handles do kernel
    n_regs = kernel.n_regs  # Número de registradores usados pelo kernel
    size_smem = kernel.metadata.shared  # Tamanho da memória compartilhada usada pelo kernel
    
    # Calcula a ocupação do kernel (quantos blocos podem ser executados por SM)
    occupancy = NUM_REGS // (n_regs * WARP_SIZE * num_warps)
    occupancy = min(occupancy, SIZE_SMEM // size_smem)
    
    # Define o número de programas (blocos) a serem executados
    num_programs = NUM_SM * occupancy
    num_programs = min(num_programs, n_rows)  # Limita ao número de linhas
    
    # Executa o kernel com os parâmetros calculados
    kernel[(num_programs, 1, 1)](y, x, x.stride(0), y.stride(0), n_rows, n_cols, BLOCK_SIZE, num_stages)
    
    # Imprime o código IR (Intermediate Representation) do kernel
    print(kernel.asm['ttir'])
    # Outras opções para depuração (comentadas):
    # print(kernel.asm['ttgir'])
    # print(triton_kernel.asm['llir'])
    # print(triton_kernel.asm['ptx'])
    # print(triton_kernel.asm['cubin'])
    
    return y  # Retorna a matriz de saída após o cálculo do softmax

In [20]:
# Importa a biblioteca time para medir o tempo de execução
import time

# Define uma semente para garantir reprodutibilidade nos resultados
torch.manual_seed(0)
# Cria uma matriz aleatória de tamanho 1823x781 na GPU
x = torch.randn(1823, 781, device=DEVICE)

# Mede o tempo de execução da implementação do softmax usando Triton
start_time = time.time()
y_triton = softmax(x)  # Executa a função softmax implementada com Triton
print("Triton time:", time.time() - start_time)  # Imprime o tempo gasto

# Mede o tempo de execução da implementação ingênua do softmax usando PyTorch
start_time = time.time()
y_torch = naive_softmax(x)  # Executa a função softmax ingênua
print("Torch time:", time.time() - start_time)  # Imprime o tempo gasto

# Imprime os resultados das duas implementações
print(y_triton)  # Resultado do Triton
print(y_torch)  # Resultado do PyTorch

# Verifica se os resultados das duas implementações são próximos (comentado para evitar erro se houver diferenças mínimas)
assert torch.allclose(y_triton, y_torch)

#loc = loc("/tmp/ipykernel_1349/820465314.py":3:0)
#loc1 = loc(unknown)
#loc12 = loc("/tmp/ipykernel_1349/820465314.py":23:37)
#loc17 = loc("/tmp/ipykernel_1349/820465314.py":27:29)
#loc26 = loc(callsite(#loc1 at #loc12))
#loc29 = loc(callsite(#loc1 at #loc17))
module {
  tt.func public @softmax_kernel(%arg0: !tt.ptr<f32> {tt.divisibility = 16 : i32} loc("/tmp/ipykernel_1349/820465314.py":3:0), %arg1: !tt.ptr<f32> {tt.divisibility = 16 : i32} loc("/tmp/ipykernel_1349/820465314.py":3:0), %arg2: i32 loc("/tmp/ipykernel_1349/820465314.py":3:0), %arg3: i32 loc("/tmp/ipykernel_1349/820465314.py":3:0), %arg4: i32 loc("/tmp/ipykernel_1349/820465314.py":3:0), %arg5: i32 loc("/tmp/ipykernel_1349/820465314.py":3:0)) attributes {noinline = false} {
    %cst = arith.constant dense<0xFF800000> : tensor<1024xf32> loc(#loc1)
    %0 = tt.get_program_id x : i32 loc(#loc2)
    %1 = tt.get_num_programs x : i32 loc(#loc3)
    scf.for %arg6 = %0 to %arg4 step %1  : i32 {
      %2 = arith.muli %arg6, %arg2 